# Training and Running of new GLM model
This Notebook is for training and running the our new GLM model that includes (for now only) deletion tokens

## Adding necessary imports
You can run this block to import necessary classes and functions

In [ ]:
from fundemental_classes.model_related.glm_model import GLMModel

model = GLMModel("./dna_bert_final", "simulated_sequences/augumented_sequence_size10000_length150_deletions0.1_nodeletionseq0.25.fasta")

## Training
We use `Bert` model to create the embeddings and train it using masking to get nicely trained model. If you want to change the size of training data, please look into `simulated_sequences` directory. \\\
*important:* Training is not necessary if you already have a trained model. You can load it using the `GLMModel`.

In [ ]:
model.train(epochs=200, batch_size=16, lr=1e-9)

## Visualization
This block is used for getting already trained model, passing the sequence to test how well the model performs.
\\\\\
*Important:* For now we pass our test cases by hand. However in the future we will generate good inputs to test how well the model functions

In [ ]:
from fundemental_classes.visualization.sequence_plotter import plot


header = ">seq76129|label=both|posAmotif=93|posBmotif=111|gaplength=10|deletions=13"
sequence = "C-CCTGA-AAGGG-GGCGATC-CATTATTATGGGTACCGAAATGGAT-CTCTTTCTGTACAAT-CTTTG-TTTG-GGAAT-AGAAGGCAACAGATATTCAATCTCGCCA-CGTACTGCACCATGAAGG-TACT-GGG-CGTAACGGGAAG"

prob_matrix = model.get_full_reconstruction_probs(sequence)

plot(header, sequence, prob_matrix, motif_length=7)